In [ ]:
import numpy as np
import os
import pandas as pd
import hdbscan
import optuna
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from hdbscan.validity import validity_index
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.cluster import AgglomerativeClustering

# Ignorar warnings
warnings.filterwarnings("ignore")
os.chdir('c:/Users/artur/OneDrive/Documents/TrabajoTesis') 

# Cargar y Limpiar
df = pd.read_csv('NeoModelos/minas_con_tiempos_puertos.csv') 
df = df[(df['RecursoPrimarioInstalacion'] == 'COBRE') | (df['RecursoMineroInstalacion'] == 'SALMUERA (LITIO)')] 

# Eliminar columnas innecesarias
cols_to_drop = ['RutEmpresa','NombreEmpresa','RecursoMineroInstalacion','TipoInstalacion',
                'TipoRecursoInstalacion','RecursoPrimarioInstalacion', 'ComunaFaena', 
                'NombreFaena', 'CategoriaFaena', 'IdFaena', 'ProvinciaInstalacion', 
                'ComunaInstalacion','NombreInstalacion','IdTipoInstalacion','IdInstalacion',
                'Norte','Este','Huso','Datum','IdEstado','Estado']

# Eliminar distancias
cols_to_drop += [c for c in df.columns if c.startswith('dist_')]
df = df.drop(columns=cols_to_drop, errors='ignore')



In [2]:
# Eliminacion por correlación alta

CORRELATION_THRESHOLD = 0.995
print(f"\n--- Columnas Redundantes (Corr > {CORRELATION_THRESHOLD}) ---")

# Columnas numéricas
df_numeric = df.select_dtypes(include=[np.number])

# Normalizar solo correlación 
scaler = RobustScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_numeric),
    columns=df_numeric.columns
)

# Calcular matriz de correlación
corr_matrix = df_scaled.corr().abs()

# Seleccionar el triángulo superior
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Identificar columnas redundantes
to_drop = [column for column in upper.columns if any(upper[column] > CORRELATION_THRESHOLD)]

if len(to_drop) > 0:
    print(f"  {len(to_drop)} columnas redundantes para eliminar.")
    # Eliminar del DataFrame original (NO normalizado)
    df = df.drop(columns=to_drop)

    print(f" Columnas eliminadas. Nueva dimensión: {df.shape}")
else:
    print(" No se encontró redundancia excesiva.")



--- Columnas Redundantes (Corr > 0.995) ---
  42 columnas redundantes para eliminar.
 Columnas eliminadas. Nueva dimensión: (10834, 18)


In [3]:
# ==== División Geográfica ====
REGIONES_NORTE = ['XV', 'I', 'II']
REGIONES_SUR   = ['III', 'IV', 'V', 'RM', 'VI', 'VII']

print(f"--- Separando Chile en 2 Modelos ---")
df_norte = df[df['RegionFaena'].isin(REGIONES_NORTE)].copy()
df_sur   = df[df['RegionFaena'].isin(REGIONES_SUR)].copy()

print(f"1. Modelo Norte (XV, I, II): {len(df_norte)} instalaciones")
print(f"2. Modelo Sur (III a VII):   {len(df_sur)} instalaciones (Zona Densa)")

print(f"--- División Geográfica ---")
print(f"Zona Norte (Antofagasta y arriba): {len(df_norte)} minas")
print(f"Zona Sur (Atacama y abajo): {len(df_sur)} minas")

--- Separando Chile en 2 Modelos ---
1. Modelo Norte (XV, I, II): 2058 instalaciones
2. Modelo Sur (III a VII):   8775 instalaciones (Zona Densa)
--- División Geográfica ---
Zona Norte (Antofagasta y arriba): 2058 minas
Zona Sur (Atacama y abajo): 8775 minas


In [4]:
def report_drop(before_cols, after_cols, label):
    removed = list(set(before_cols) - set(after_cols))
    print(f"\n {label}: removed {len(removed)} columns")
    if removed:
        print(f"   → {removed}")
    print(f"   Columns before: {len(before_cols)} | after: {len(after_cols)}")


In [5]:
def preparar_datos(df_region):
    """
    1. One-hot encoding
    2. Selección numérica
    3. Filtro: drop dummies con freq <1% o >99% (Frecuencia Extrema)
    4. Escalado robusto
    """
    # One Hot Encoding
    df_encoded = pd.get_dummies(df_region, columns=['ProvinciaFaena'], drop_first=True, dtype=int)
    
    # Selección inicial de numéricas
    df_model = df_encoded.select_dtypes(include=[np.number]).dropna()
    
    # Identificar columnas dummies creadas
    dummy_cols = [c for c in df_model.columns if c.startswith("ProvinciaFaena_")]
    
    # Frecuencia Extrema
    before = df_model.columns.tolist()
    low_freq_threshold = 0.02
    high_freq_threshold = 0.98
    
    drop_freq = []
    for col in dummy_cols:
        freq = df_model[col].mean()
        if freq < low_freq_threshold or freq > high_freq_threshold:
            drop_freq.append(col)
            
    if drop_freq:
        df_model = df_model.drop(columns=drop_freq)
        
    report_drop(before, df_model.columns.tolist(), "Filtro de Frecuencia")

    # Escalado Final
    print(f" Dimensiones finales antes de escalar: {df_model.shape}")
    
    # Usamos RobustScaler para manejar outliers en tiempos de viaje
    scaler = RobustScaler()
    X = scaler.fit_transform(df_model.values)

    return X, df_model.columns.tolist(), df_model.index



In [6]:
def obtener_grupos_variables(feature_names):
    geo_idxs = [i for i, c in enumerate(feature_names) if c in ['Latitud', 'Longitud']]
    log_idxs = [i for i, c in enumerate(feature_names) if c.startswith('time_') or c.startswith('Tiempo_Prt_')]
    cat_idxs = [i for i, c in enumerate(feature_names) if c.startswith('ProvinciaFaena_')]
    other_idxs = [i for i in range(len(feature_names)) if i not in geo_idxs + log_idxs + cat_idxs]

    return geo_idxs, log_idxs, cat_idxs, other_idxs


In [7]:
def crear_objective(X_scaled, geo_idxs, log_idxs, cat_idxs, other_idxs, ranges, selection_method):

    def objective(trial):
        # Pesos
        w_geo = 1.0
        w_log = trial.suggest_float("w_log", *ranges['w_log'])
        w_cat = trial.suggest_float("w_cat", *ranges['w_cat'])
        w_oth = trial.suggest_float("w_oth", *ranges['w_oth'])

        Xw = X_scaled.copy()
        if geo_idxs: Xw[:, geo_idxs] *= w_geo
        if log_idxs: Xw[:, log_idxs] *= w_log
        if cat_idxs: Xw[:, cat_idxs] *= w_cat
        if other_idxs: Xw[:, other_idxs] *= w_oth

        # PCA
        max_comp = min(20, Xw.shape[1])
        n_comp = trial.suggest_int("n_components", 5, max_comp)
        Xp = PCA(n_components=n_comp, random_state=42).fit_transform(Xw)

        # HDBSCAN
        min_cluster = trial.suggest_int("min_cluster", *ranges['min_cluster'])
        min_samples = trial.suggest_int("min_samples", *ranges['min_samples'])

        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=min_cluster,
            min_samples=min_samples,
            metric='euclidean',
            gen_min_span_tree=True,
            cluster_selection_method=selection_method
        ).fit(Xp)

        labels = clusterer.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters < 8: return -1.0

        # Métricas
        try:
            dbcv = validity_index(Xp, labels)
            
            # Guardar el DBCV puro antes de castigos
            trial.set_user_attr("dbcv_raw", dbcv) 

            # Silhouette
            mask = labels != -1
            if mask.sum() > n_clusters:
                sil = silhouette_score(Xp[mask], labels[mask])
            else:
                sil = -1.0
            trial.set_user_attr("silhouette", sil)

        except:
            return -1.0

        # Penalizaciones
        score = dbcv
        
        # Castigo por ruido
        ruido = (labels == -1).mean()
        if ruido > 0.25: 
            score -= 0.30
            
        # Castigo por exceso de clusters
        if n_clusters > 60:
            score -= 0.4

        return score

    return objective

In [8]:
def entrenar_modelo_regional(df_region, nombre_zona, n_trials=30, param_ranges=None, selection_method='leaf'):

    defaults = {
        'w_log': (1.0, 3.5),
        'w_cat': (1.0, 2.5),
        'w_oth': (0.5, 1.5),
        'min_cluster': (15, 40), 
        'min_samples': (15, 30),
        'n_components': (5, 20) 
    }

    ranges = defaults.copy()
    if param_ranges:
        ranges.update(param_ranges)

    print(f"\n Optimización DBCV para {nombre_zona} | Método: {selection_method}")


    X_scaled, feature_names, valid_idx = preparar_datos(df_region)
    geo, log, cat, oth = obtener_grupos_variables(feature_names)

    # Objective 
    objective = crear_objective(X_scaled, geo, log, cat, oth, ranges, selection_method)

    # Suprimir logs
    optuna.logging.set_verbosity(optuna.logging.WARNING) 

    # Optuna
    study = optuna.create_study(direction="maximize")
    
    # show_progress_bar=True
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    best_trial = study.best_trial
    
    # Métricas guardadas
    sil = best_trial.user_attrs.get("silhouette", "N/A")
    dbcv_raw = best_trial.user_attrs.get("dbcv_raw", "N/A") 

    print(f"   Mejor Score (Castigado): {study.best_value:.4f}")
    print(f"   DBCV Real (Sin castigo): {dbcv_raw}") 
    print(f"   Silhouette: {sil}")
    print(f"   Parámetros: {study.best_params}")

    # Reconstrucción final
    bp = best_trial.params

    Xw = X_scaled.copy()
    if geo: Xw[:, geo] *= 1.0
    if log: Xw[:, log] *= bp['w_log']
    if cat: Xw[:, cat] *= bp['w_cat']
    if oth: Xw[:, oth] *= bp['w_oth']

    pca_final = PCA(n_components=bp['n_components'], random_state=42).fit_transform(Xw)

    final_clusterer = hdbscan.HDBSCAN(
        min_cluster_size=bp['min_cluster'],
        min_samples=bp['min_samples'],
        metric='euclidean',
        cluster_selection_method=selection_method
    ).fit(pca_final)


    return final_clusterer.labels_, valid_idx

In [9]:

# Configuración para el Norte 
config_norte = {
    'min_cluster': (30, 42),    
    'min_samples': (32, 45),    
}

# Configuración para el Sur 
config_sur = {
    'min_cluster': (30, 45),    
    'min_samples': (25, 35),    
}

# Entrenar Norte
labels_norte, idx_norte = entrenar_modelo_regional(
    df_norte, 
    "ZONA NORTE", 
    n_trials=60, 
    param_ranges=config_norte,
    selection_method='leaf'  
)

# Entrenar Sur
labels_sur, idx_sur = entrenar_modelo_regional(
    df_sur, 
    "ZONA SUR", 
    n_trials=60, 
    param_ranges=config_sur,
    selection_method='leaf' 
)


 Optimización DBCV para ZONA NORTE | Método: leaf

 Filtro de Frecuencia: removed 1 columns
   → ['ProvinciaFaena_PARINACOTA']
   Columns before: 21 | after: 20
 Dimensiones finales antes de escalar: (2054, 20)


  0%|          | 0/60 [00:00<?, ?it/s]

   Mejor Score (Castigado): 0.4504
   DBCV Real (Sin castigo): 0.4503819813727218
   Silhouette: 0.6400891483990732
   Parámetros: {'w_log': 1.151919623529112, 'w_cat': 2.213070128973989, 'w_oth': 0.9266964836806384, 'n_components': 12, 'min_cluster': 38, 'min_samples': 44}

 Optimización DBCV para ZONA SUR | Método: leaf

 Filtro de Frecuencia: removed 11 columns
   → ['ProvinciaFaena_VALPARAISO', 'ProvinciaFaena_QUILLOTA', 'ProvinciaFaena_SANTIAGO', 'ProvinciaFaena_MARGA MARGA', 'ProvinciaFaena_TALCA', 'ProvinciaFaena_CORDILLERA', 'ProvinciaFaena_MELIPILLA', 'ProvinciaFaena_COLCHAGUA', 'ProvinciaFaena_TALAGANTE', 'ProvinciaFaena_MAIPO', 'ProvinciaFaena_CURICO']
   Columns before: 36 | after: 25
 Dimensiones finales antes de escalar: (8759, 25)


  0%|          | 0/60 [00:00<?, ?it/s]

   Mejor Score (Castigado): 0.0128
   DBCV Real (Sin castigo): 0.3128283109288762
   Silhouette: 0.5895769674361407
   Parámetros: {'w_log': 1.615889830065293, 'w_cat': 1.9905313056027598, 'w_oth': 0.7831644769453432, 'n_components': 17, 'min_cluster': 40, 'min_samples': 28}


In [10]:
# === Unificación de Modelos ====
df['cluster_raw'] = -2 

# Asignar Norte 
df.loc[idx_norte, 'cluster_raw'] = labels_norte

# Asignar Sur 
max_id_norte = max(set(labels_norte) - {-1}) if len(set(labels_norte) - {-1}) > 0 else 0
offset = max_id_norte + 1

# Función lambda para desplazar solo si no es ruido (-1)
shifted_labels_sur = [x + offset if x >= 0 else -1 for x in labels_sur]
df.loc[idx_sur, 'cluster_raw'] = shifted_labels_sur

print("\n == Modelos unificados ==")
print(f"Total Clusters Brutos: {len(set(df['cluster_raw']) - {-1, -2})}")


 == Modelos unificados ==
Total Clusters Brutos: 55


In [11]:
# == Configuración ==
col_analisis = 'cluster_raw'
col_final = 'cluster_final'
REGIONES_NORTE = ['XV', 'I', 'II']
# Probamos radios (grados) para ver cuál compacta mejor sin mezclar valles distintos
RANGO_EPS = np.linspace(0.15, 0.40, 40) 

def optimizar_fusion_zona(df_zona, zona_name, start_id=0):
    print(f"\n Optimizando Fusión (Aglomerativa) para: {zona_name}...")
    
    # Preparar datos locales (sin ruido de proceso)
    df_eval = df_zona[df_zona[col_analisis] != -2].copy()
    if len(df_eval) == 0:
        return {}, start_id, 0

    # Preprocesamiento local para evaluar calidad interna
    df_encoded = pd.get_dummies(df_eval, columns=['ProvinciaFaena'], drop_first=True, dtype=int)
    X_feat = df_encoded.select_dtypes(include=[np.number]).values
    X_scaled = RobustScaler().fit_transform(X_feat)
    
    # Calcular Centroides
    unique_labels = set(df_eval[col_analisis]) - {-1}
    if not unique_labels:
        return {}, start_id, 0

    centroids = []
    label_map_list = []
    
    for label in unique_labels:
        data = df_eval[df_eval[col_analisis] == label][['Latitud', 'Longitud']].values
        centroid = data.mean(axis=0)
        centroids.append(centroid)
        label_map_list.append(label)
        
    centroids = np.array(centroids)
    
    # Bucle de Optimización de Radio (Distance Threshold)
    best_score = -2.0
    best_eps = 0.0
    best_mapping = None
    best_n_clusters = 0
    
    for eps in RANGO_EPS:
        # Usamos AgglomerativeClustering
        agg_cluster = AgglomerativeClustering(
            n_clusters=None, 
            distance_threshold=eps, 
            metric='euclidean',
            linkage='complete' 
        )
        try:
            merged_ids = agg_cluster.fit_predict(centroids)
        except ValueError:
            # Si solo hay 1 cluster o error interno
            merged_ids = np.zeros(len(centroids), dtype=int)

        # Mapeo temporal
        mapping = {old: new for old, new in zip(label_map_list, merged_ids)}
        mapping[-1] = -1
        
        # Evaluar
        labels_temp = df_eval[col_analisis].map(mapping).values
        n_clust = len(set(labels_temp) - {-1})
        
        if n_clust > 1:
            try:
                score = validity_index(X_scaled, labels_temp, metric='euclidean')
            except:
                score = -1.0
        else:
            score = -1.0
            
        # Guardar mejor
        if score > best_score:
            best_score = score
            best_eps = eps
            best_mapping = mapping
            best_n_clusters = n_clust
            
        
    print(f"MEJOR {zona_name}: Radio {best_eps:.3f}° | DBCV {best_score:.4f} | Clusters: {best_n_clusters}")
    
    # Generar Mapeo Global
    final_map = {}
    if best_mapping:
        for old_id, new_id in best_mapping.items():
            if new_id == -1:
                final_map[old_id] = -1
            else:
                final_map[old_id] = new_id + start_id
    else:
        for i, old_id in enumerate(unique_labels):
            final_map[old_id] = i + start_id

    used_ids = [v for v in final_map.values() if v != -1]
    next_start = (max(used_ids) + 1) if used_ids else start_id
    
    return final_map, next_start, best_score

# == Ejecución Principal ==

# Separar DataFrame
print("--- Separando zonas para fusión ---")
df_norte = df[df['RegionFaena'].isin(REGIONES_NORTE)]
df_sur = df[~df['RegionFaena'].isin(REGIONES_NORTE)]

# Optimizar Norte
mapa_norte, next_id, score_norte = optimizar_fusion_zona(df_norte, "ZONA NORTE", start_id=0)

# Optimizar Sur
mapa_sur, _, score_sur = optimizar_fusion_zona(df_sur, "ZONA SUR", start_id=next_id)

# Aplicar al DataFrame
full_mapping = {**mapa_norte, **mapa_sur}
full_mapping[-1] = -1
full_mapping[-2] = -2

df[col_final] = df[col_analisis].map(full_mapping).fillna(-1).astype(int)

# === Evaluación Global Final ===
print("\n Métricas Finales del Modelo Aglomerativo:")
df_final_eval = df[df[col_final] != -2].copy()

# Escalar datos globales
df_enc = pd.get_dummies(df_final_eval, columns=['ProvinciaFaena'], drop_first=True, dtype=int)
X_final = RobustScaler().fit_transform(df_enc.select_dtypes(include=[np.number]).values)
labels_final = df_final_eval[col_final].values

# Cálculo de Métricas
n_final = len(set(labels_final) - {-1})
ruido_final = (labels_final == -1).mean()

try:
    sil_final = silhouette_score(X_final[labels_final != -1], labels_final[labels_final != -1])
except:
    sil_final = -1.0

try:
    dbcv_final = validity_index(X_final, labels_final, metric='euclidean')
except:
    dbcv_final = 0.0

print(f"   • Total Clusters: {n_final}")
print(f"   • Ruido Global:   {ruido_final:.1%}")
print(f"   • Silhouette:     {sil_final:.4f}")
print(f"   • DBCV Global:    {dbcv_final:.4f}")

--- Separando zonas para fusión ---

 Optimizando Fusión (Aglomerativa) para: ZONA NORTE...
MEJOR ZONA NORTE: Radio 0.150° | DBCV 0.4026 | Clusters: 11

 Optimizando Fusión (Aglomerativa) para: ZONA SUR...
MEJOR ZONA SUR: Radio 0.246° | DBCV 0.2055 | Clusters: 29

 Métricas Finales del Modelo Aglomerativo:
   • Total Clusters: 40
   • Ruido Global:   37.2%
   • Silhouette:     0.5553
   • DBCV Global:    0.2837


In [12]:
def calcular_metricas_zona(df_zona, col_cluster, nombre_zona):
    """
    Calcula Silhouette y DBCV para un subconjunto específico de datos.
    """
    print(f"\n == Evaluando: {nombre_zona} ==")
    
    # Filtrar ruido
    df_eval = df_zona[df_zona[col_cluster] != -2].copy()
    
    if len(df_eval) == 0:
        print("No hay datos en esta zona.")
        return

    # Preprocesamiento
    df_encoded = pd.get_dummies(df_eval, columns=['ProvinciaFaena'], drop_first=True, dtype=int)
    X_eval = df_encoded.select_dtypes(include=[np.number]).values
    X_scaled = RobustScaler().fit_transform(X_eval)
    
    labels = df_eval[col_cluster].values
    n_clusters = len(set(labels) - {-1})
    
    # Métricas
    # Silhouette
    mask_clustered = labels != -1
    if np.sum(mask_clustered) > 2 and n_clusters > 1:
        sil = silhouette_score(X_scaled[mask_clustered], labels[mask_clustered], metric='euclidean')
    else:
        sil = -1.0
        
    # DBCV 
    try:
        if n_clusters >= 1:
            dbcv = validity_index(X_scaled, labels, metric='euclidean')
        else:
            dbcv = 0.0
    except Exception as e:
        print(f"   Error DBCV: {e}")
        dbcv = 0.0

    # 4. Reporte
    print(f"   • Total Clusters: {n_clusters}")
    print(f"   • Ruido:          {(np.sum(labels == -1) / len(labels)):.1%}")
    print(f"   ---------------------------")
    print(f"   • Silhouette:     {sil:.4f}")
    print(f"   • DBCV:           {dbcv:.4f}")

# --- EJECUCIÓN POR ZONAS ---
# Definir las regiones para separar el DataFrame FINAL
REGIONES_NORTE = ['XV', 'I', 'II']

# Crear los subconjuntos del DF final 
df_final_norte = df[df['RegionFaena'].isin(REGIONES_NORTE)].copy()
df_final_sur   = df[~df['RegionFaena'].isin(REGIONES_NORTE)].copy()

# Evaluar Norte Final
calcular_metricas_zona(df_final_norte, col_final, "ZONA NORTE (Final)")

# Evaluar Sur Final
calcular_metricas_zona(df_final_sur, col_final, "ZONA SUR (Final)")

# Evaluar Global 
calcular_metricas_zona(df, col_final, "GLOBAL (Chile)")


 == Evaluando: ZONA NORTE (Final) ==
   • Total Clusters: 11
   • Ruido:          22.8%
   ---------------------------
   • Silhouette:     0.6257
   • DBCV:           0.4345

 == Evaluando: ZONA SUR (Final) ==
   • Total Clusters: 29
   • Ruido:          40.6%
   ---------------------------
   • Silhouette:     0.5325
   • DBCV:           0.2662

 == Evaluando: GLOBAL (Chile) ==
   • Total Clusters: 40
   • Ruido:          37.2%
   ---------------------------
   • Silhouette:     0.5553
   • DBCV:           0.2837


In [13]:
# --- Visualización ---
import plotly.express as px

# Filtrar ruido para el mapa
plot_data = df[df[col_final] != -2].copy()
plot_data['Cluster_ID'] = plot_data[col_final].astype(str)
plot_data = plot_data.sort_values(col_final)

fig = px.scatter_mapbox(
    plot_data,
    lat="Latitud",
    lon="Longitud",
    color="Cluster_ID",
    color_discrete_map={'-1': 'lightgray'}, 
    hover_name="Cluster_ID",
    hover_data=["ProvinciaFaena"],
    zoom=5,
    center={"lat": -28.0, "lon": -69.5},
    title="Modelo Dual (Norte/Sur) con Fusión de Radio",
    height=900
)

fig.update_layout(mapbox_style="carto-positron")
fig.show()